In [ ]:
# import streamlit as st
import numpy as np
import pandas as pd
from IPython.display import display

pd.set_option('display.max_colwidth', None)
pd.set_option('display.width', 260)
pd.set_option('display.max_columns', 200)
pd.set_option('display.max_rows', 120)

pref='../'

import pickle

from catboost import CatBoostClassifier, Pool
# import concurrent.futures
import time

import sys
sys.path.append(
    'C:\\Users\\peter\\Documents\\MyProjects\\PyProj\\Trav\\spel')

import typ_copy as tp
import travdata as td


Läs in data och tvätta

In [ ]:
###############################################################################
#                           Läs in all data och tvätta                        #
###############################################################################
def läs_in_data(förbered=True, lägg_till=True):
    v75 = td.v75(pref=pref)
    _ = v75.förbered_data(missing_num=False)  # num hanteras av catboost
    df = v75.test_lägg_till_kolumner()
    return df,v75

### Vi förbereder ensemble stacking classifier med 3 modeller och en meta-modell

In [ ]:
# import module for ensamble stacking classifier
from sklearn.ensemble import StackingClassifier

In [ ]:
# models to be used in stacking
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.neighbors import KNeighborsClassifier

# meta-model to be used in final classifier
from sklearn.linear_model import RidgeClassifier

# create models
rf = RandomForestClassifier(n_estimators=100, random_state=42)
et = ExtraTreesClassifier(n_estimators=100, random_state=42)
knn = KNeighborsClassifier(n_neighbors=5)
lr = RidgeClassifier()  # meta-model

Set up stacking med TimeSeriesSplit

In [ ]:
# set up time series split
from sklearn.model_selection import TimeSeriesSplit
tscv = TimeSeriesSplit(n_splits=5)

In [ ]:
# set up the stacking classifier
stacker = StackingClassifier(estimators=[('rf', rf), ('et', et), ('knn', knn)],
                                final_estimator=RidgeClassifier(),  # meta-model
                                cv=tscv,
                                n_jobs=5)

In [ ]:
df, v75 = läs_in_data()

In [ ]:
df.head(0)

Remove features not used here

In [ ]:
df_work,enc = v75.förbered_data(missing_num=True, 
                            missing_cat=True, 
                            # cardinality_list=['häst', 'bana', 'kusk', 'h1_kusk', 'h2_kusk', 'h3_kusk', 'h4_kusk', 'h5_kusk', 'h1_bana', 'h2_bana', 'h3_bana', 'h4_bana', 'h5_bana'], 
                            target_encode_list=['häst', 'bana', 'kusk', 'kön','h1_kusk', 'h2_kusk', 'h3_kusk', 'h4_kusk', 'h5_kusk', 'h1_bana', 'h2_bana', 'h3_bana', 'h4_bana', 'h5_bana'],
                            remove_mer=[])
df_work.head(3)

In [ ]:
# stacker fit
stacker.fit(df_work.drop(['datum','avd','y'], axis=1), df_work['y'])
